In [2]:
from imports import *
import ast
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import torch.nn as nn
import re

In [3]:
og_df = pd.read_csv('games.csv')
og_df.head()

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,3,Undertale,"Sep 15, 2015","['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,4,Hollow Knight,"Feb 24, 2017",['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K


In [4]:
temp = pd.read_csv('vgsales.csv')
temp = temp[['Genre', 'Name']]
temp.head()

,Genre,Name
0,Sports,Wii Sports
1,Platform,Super Mario Bros.
2,Racing,Mario Kart Wii
3,Sports,Wii Sports Resort
4,Role-Playing,Pokemon Red/Pokemon Blue


In [ ]:
def concate(row):
    valid = ['Adventure', 'Shooter', 'RPG', 'Strategy', 'Simulator']
    row_concat = []
    for genre in row:
        genre = genre.strip('[]" \'')
        if genre in valid:
            row_concat.append(genre)
    return "-".join(row_concat)

def clean(review):
    review = review.lower()
    review = re.sub(r'[^a-zA-Z0-9 ]', '', review)
    return review

reviews = og_df[['Title', 'Genres', 'Reviews']].merge(temp, left_on = 'Title', right_on= 'Name', how = 'inner')
reviews['Reviews'] = reviews['Reviews'].apply(ast.literal_eval)
exploded_reviews = pd.DataFrame(reviews.set_index('Genre')['Reviews'].explode()).reset_index()
df = exploded_reviews.dropna()
df['Reviews'] = df['Reviews'].apply(clean)
df

,Genre,Reviews
0,Misc,minecraft is what you make of it unfortunately...
1,Misc,muito bom vc pode construir usar sua imaginao ...
2,Misc,jogo incrvel muito divertido e com amigos fica...
3,Misc,baby gammeeee ...
4,Misc,
...,...,...
8059,Action,pretty average lego game but it was my childhood
8060,Action,pog lego game
8061,Action,pretty decent lego game loved the attention to...
8062,Action,class


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

le = LabelEncoder()
df['genre_encoded'] = le.fit_transform(df['Genre'])

X_train, X_test, y_train, y_test = train_test_split(df['Reviews'], df['Genre'], test_size=0.2)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000)),
    ('nb', MultinomialNB()) 
])

pipeline.fit(X_train, y_train)

y_test_pred = pipeline.predict(X_test)

print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

Test Classification Report:
              precision    recall  f1-score   support

      Action       0.49      0.97      0.65       506
   Adventure       1.00      0.27      0.43       113
    Fighting       1.00      0.02      0.04        52
        Misc       1.00      0.36      0.53        66
    Platform       0.86      0.55      0.67       200
      Puzzle       1.00      0.05      0.10        19
      Racing       0.00      0.00      0.00        40
Role-Playing       0.79      0.60      0.68       270
     Shooter       0.88      0.65      0.74       271
  Simulation       0.00      0.00      0.00        17
      Sports       1.00      0.54      0.70        26
    Strategy       0.00      0.00      0.00        33

    accuracy                           0.63      1613
   macro avg       0.67      0.34      0.38      1613
weighted avg       0.71      0.63      0.59      1613



c:\Users\echen\miniforge3\envs\dsc80\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\echen\miniforge3\envs\dsc80\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\echen\miniforge3\envs\dsc80\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import joblib

joblib.dump(pipeline, 'genre_classifier_model.joblib')
# loaded_model = joblib.load('genre_classifier_model.joblib')

['genre_classifier_model.joblib']